In [1]:
! pip install rarfile

In [2]:
import os
import pandas as pd
import glob
import rarfile
import openpyxl
import zipfile
from datetime import datetime

In [8]:
# Extract RAR file
rar_file_path = 'pgcb.rar'  # Define rar file path

# Extract the contents of the rar file
extracted_files_dir = 'extracted_files'
with rarfile.RarFile(rar_file_path, 'r') as rar_ref:
    rar_ref.extractall(extracted_files_dir)  # Extract all files into a folder

In [ ]:
# # Define your zip file path
# zip_file_path = 'path_to_your_zip_file.zip'  # Replace with your zip file path

# # Extract the contents of the zip file
# extracted_files_dir = 'extracted_files'
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extracted_files_dir)  # Extract all files into a folder

In [10]:
# List all xlsm files extracted
xlsm_files = [file for file in os.listdir(extracted_files_dir) if file.endswith('.xlsm')]

# Initialize a list to store extracted data from all files
all_data = []

# Process each xlsm file
for xlsm_file in xlsm_files:
    file_path = os.path.join(extracted_files_dir, xlsm_file)

    # Load the xlsm file
    workbook = openpyxl.load_workbook(file_path, read_only=True)

    # Assuming the data is in a specific sheet, replace 'P3' with your sheet name
    sheet = workbook['P3']

    # Initialize variables for storing found data
    sub_station = None
    load_mw = None
    hour = None
    date = None

    # Search for 'Bagerhat' and 'Date:'
    found_bagerhat = False
    for row in sheet.iter_rows():
        for cell in row:
            if cell.value == 'Bagerhat':
                found_bagerhat = True
                sub_station = sheet.cell(row=cell.row, column=cell.column).value
                load_mw = sheet.cell(row=cell.row, column=cell.column + 1).value
                hour = sheet.cell(row=cell.row, column=cell.column + 2).value
                break
            elif cell.value == 'Date:':
                date = sheet.cell(row=cell.row, column=cell.column + 1).value

        if found_bagerhat and date:
            # Extract date and time parts
            date_part = date.strftime('%d-%m-%Y')  # Format date as '14-02-2020'
            hour_value = hour.strftime('%H:%M:%S')  # Format hour as '12:00:00'

            # Combine date and hour into a datetime object
            combined_datetime = datetime.strptime(f'{date_part} {hour_value}', '%d-%m-%Y %H:%M:%S')

            # Append the data to all_data
            all_data.append([combined_datetime, sub_station, load_mw, hour])
            break

    # Close the workbook
    workbook.close()

In [11]:
# Sort all_data by the first column (datetime)
all_data.sort(key=lambda x: x[0])

# Create a final table with all extracted data
final_table = [['DateTime', 'Sub-station', 'Load (MW)']]
final_table.extend([[row[0], row[1], row[2]] for row in all_data])

# Print or process final_table as needed
print(final_table)

[['DateTime', 'Sub-station', 'Load (MW)'], [datetime.datetime(2020, 2, 14, 19, 0), 'Bagerhat', 62], [datetime.datetime(2020, 3, 15, 19, 30), 'Bagerhat', 75], [datetime.datetime(2020, 3, 16, 19, 30), 'Bagerhat', 74]]


In [12]:
# Save the final_table to an Excel file using openpyxl
from openpyxl import Workbook

wb = Workbook()
ws = wb.active

for row in final_table:
    ws.append(row)

output_excel_file = 'output.xlsx'
wb.save(output_excel_file)

In [13]:
# Clean up: remove the extracted files folder if desired
import shutil
shutil.rmtree(extracted_files_dir)

print(f"Data extracted and saved to '{output_excel_file}'.")

Data extracted and saved to 'output.xlsx'.
